# Don't run this code
I would strongly advise against running this code as it takes several minutes to process all of the data, as well as requiring a few datasets that aren't in shared data. I've included it for documentation purposes and such but I made much more simplified summary datasets from the data that are saved to shared data. 

This code creates four datasets:
1. County_Building_Counts: Number of reported commercial buildings in each county
2. County_Building_Counts_By_Type: Number of reported commercial buildings in each county, further subdivided by building type (Hospitality, Retail, etc.)
3. County_Building_Areas: Total area of buildings in each county
4. County_Building_Areas_Adjusted: Total area of buildings in each county, divided by land area of county (Building density). Probably the best one to use. 

In [57]:
import pandas as pd
import os
import geopandas as gpd

current_dir = os.getcwd()

In [7]:
west_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'West.xlsb'), sheet_name="County")
print("Processed west!")
midwest_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'Midwest.xlsb'), sheet_name="County")
print("Processed midwest!")
southatlantic_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'SouthAtlantic.xlsb'), sheet_name="County")
print("Processed south Atlantic!")
southcentral_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'SouthCentral.xlsb'), sheet_name="County")
print("Processed south central!")
northeast_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'Northeast.xlsb'), sheet_name="County")
print("Processed northeast!")

Processed west!
Processed midwest!
Processed south Atlantic!
Processed south central!
Processed northeast!


534677

In [21]:
df = pd.concat([west_df, midwest_df, southatlantic_df, southcentral_df, northeast_df])
len(df)

2246485

In [23]:
copy_df = df.copy()
copy_df.head()

,Unnamed: 0,city_name,Unnamed: 2,countyname,statefp,statecode,bldg_count,reported_propertytype,reported_propertysubtype,rentablebuildingarea_mean,...,cbecs,Unnamed: 15,area_sum,Unnamed: 17,placeid,countyid,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Hospitality,Hotel,5200,...,Hotel,NaN,5200,NaN,NaN,2013,NaN,NaN,NaN,NaN
1,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Hospitality,Motel,5200,...,Motel or inn,NaN,5200,NaN,NaN,2013,NaN,NaN,NaN,NaN
2,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Multi-Family,Apartments,5000,...,Apartments,NaN,5000,NaN,NaN,2013,NaN,NaN,NaN,NaN
3,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Specialty,Lodge/Meeting Hall,16700,...,Other public assembly,NaN,16700,NaN,NaN,2013,NaN,NaN,NaN,NaN
4,NaN,Unalaska,NaN,Aleutians West Census Area,2,AK,1,Multi-Family,Apartments,15200,...,Apartments,NaN,15200,NaN,280770.0,2016,NaN,NaN,NaN,NaN


In [36]:
grouped = copy_df[["statecode", "countyname", "bldg_count"]].groupby(["statecode", "countyname"]).agg("sum").reset_index()
sorted_grouped = grouped.sort_values("bldg_count", ascending=False).head(10)

In [37]:
sorted_grouped.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts.csv'), index=False)

In [43]:
grouped_use = copy_df[["statecode", "countyname", "reported_propertytype", "bldg_count"]].groupby(["statecode", "countyname", "reported_propertytype"]).agg("sum").reset_index()
# grouped_stories.sort_values("stories", ascending=False).head(20)
grouped_use.head(10)

,statecode,countyname,reported_propertytype,bldg_count
0,AK,Aleutians East Borough,Hospitality,2
1,AK,Aleutians East Borough,Multi-Family,1
2,AK,Aleutians East Borough,Specialty,1
3,AK,Aleutians West Census Area,Multi-Family,1
4,AK,Aleutians West Census Area,Office,2
...,...,...,...,...
23731,WY,Weston County,Industrial,1
23732,WY,Weston County,Multi-Family,3
23733,WY,Weston County,Office,8
23734,WY,Weston County,Retail,18


In [49]:
grouped_area = copy_df[["statecode", "countyname", "area_sum"]].groupby(["statecode", "countyname"]).agg("sum").reset_index()
grouped_area.sort_values("area_sum", ascending=False)

,statecode,countyname,area_sum
204,CA,Los Angeles County,3450593400
709,IL,Cook County,2147458100
2621,TX,Harris County,2001257800
2010,NY,New York County,1822910500
2577,TX,Dallas County,1505328800
...,...,...,...
1849,NE,Hayes County,1400
1545,MS,Issaquena County,1400
377,FL,St Lucie County,1100
2857,VA,Hampton City,1000


In [61]:
def translate_state(state_name):
    state_abbreviations = {
        'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'
    }

    return state_abbreviations[state_name]

In [73]:
# Adjust for area differences
land_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'LandArea.xls'), dtype=str)
land_df = land_df[["STCOU", "LND110210D"]] # LND110210D is Land Area of counties in 2010 (not including water), STCOU is state fips and county fips
land_df["STATE_FIPS"] = land_df["STCOU"].str[:-3] # Extract just state fips
land_df["CNTY_FIPS"] = land_df["STCOU"].str[-3:] # Extract just county fips

counties = gpd.read_file(os.path.join(current_dir, '..', 'SharedData', 'UScounties', 'UScounties.shp')).drop("geometry", axis=1) # County dataset

merged1 = pd.merge(land_df, counties, on=["STATE_FIPS", "CNTY_FIPS"], how="inner", validate="1:1")
merged1['statecode'] = merged1['STATE_NAME'].apply(translate_state)
merged1 = merged1.rename(columns={"NAME": "COUNTY"})
grouped_area = grouped_area.rename(columns={"countyname": "COUNTY"})
merged_df = pd.merge(merged1, grouped_area, on=['statecode'], suffixes=('_df1', '_df2'))
def merge_county(df1_county, df2_county):
    return df2_county.startswith(df1_county)
merged_df = merged_df[merged_df.apply(lambda x: merge_county(x['COUNTY_df1'], x['COUNTY_df2']), axis=1)]

merged_df["bldg_area_per_sq_mile"] = merged_df["area_sum"] / merged_df["LND110210D"].astype(float)
merged_df.sort_values("bldg_area_per_sq_mile", ascending=False)

adjusted_grouped_area = merged_df[["STATE_NAME", "statecode", "STATE_FIPS", "COUNTY_df1", "CNTY_FIPS", "area_sum", "LND110210D", "bldg_area_per_sq_mile"]]
adjusted_grouped_area = adjusted_grouped_area.rename(columns={"LND110210D": "LandAreaSqMiles", "COUNTY_df1":"COUNTY", "statecode":"STATECODE", "area_sum":"total_bldg_area"})
adjusted_grouped_area.head(10)

,STATE_NAME,STATECODE,STATE_FIPS,COUNTY,CNTY_FIPS,total_bldg_area,LandAreaSqMiles,bldg_area_per_sq_mile
0,Alabama,AL,01,Autauga,001,6993000,594.44,11764.013189
68,Alabama,AL,01,Baldwin,003,60335200,1589.78,37951.917875
136,Alabama,AL,01,Barbour,005,6098300,884.88,6891.668927
204,Alabama,AL,01,Bibb,007,1386100,622.58,2226.380545
272,Alabama,AL,01,Blount,009,2821200,644.78,4375.445889
340,Alabama,AL,01,Bullock,011,553000,622.81,887.911241
408,Alabama,AL,01,Butler,013,2858700,776.83,3679.955717
476,Alabama,AL,01,Calhoun,015,23999100,605.87,39610.972651
544,Alabama,AL,01,Chambers,017,6910000,596.53,11583.658827
612,Alabama,AL,01,Cherokee,019,1849000,553.7,3339.353440


In [74]:
grouped.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts.csv'), index=False)
grouped_use.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts_By_Type.csv'), index=False)
grouped_area.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Areas.csv'), index=False)
adjusted_grouped_area.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Areas_Adjusted.csv'), index=False)